# [IAPR][iapr]: Lab 1 ‒  Image segmentation


**Group ID:** xx

**Author 1 (sciper):** Student Name 1 (xxxxx)  
**Author 2 (sciper):** Student Name 2 (xxxxx)   
**Author 3 (sciper):** Student Name 3 (xxxxx)   

**Release date:** 05.03.2025   
**Due date:** 19.03.2025 (11:59 pm)


## Key Submission Guidelines:
- **Before submitting your notebook, <span style="color:red;">rerun</span> it from scratch!** Go to: `Kernel` > `Restart & Run All`  
- **Only groups of three will be accepted**, except in exceptional circumstances.  
- **You are not allowed to use any libraries** other than those provided in this notebook.  
- **Failure to follow these guidelines may result in point deductions** during grading.  


[iapr]: https://github.com/LTS5/iapr2025

In [ ]:
# Check is at least python 3.9
import sys 
assert (sys.version_info.major == 3) and (sys.version_info.minor >= 9)

In [ ]:
# Install required packages
!pip install numpy -q
!pip install matplotlib -q
!pip install scikit-image -q
!pip install pillow -q

In [ ]:
# Import main packages
from utils.lab_01_utils import *
from skimage.color import rgb2hsv, rgb2gray
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects, binary_dilation


---
# Introduction


In this lab, we will be working with histological data. Histological images are created from tissue samples that are digitized using high-resolution scanners. Without preparation, human tissues lack visual contrast and have poor tissue differentiations. To overcome this issue, clinical institutes use chemical staining to artificially enhance contrast. The most famous one is called HE (hematoxylin and eosin) staining. Hematoxylin stains tissue nuclei with a deep purple while eosin focuses more on extracellular matrix components with a pink stain.

Before running the following code make sure that the images are located as follows:

```code
├── labs
|   ├── utils
|   |   └── lab_01_utils.py
|   └── lab_01_iapr.ipynb
└── data
    └── data_lab_01
        ├── tcga_crc_example.png
        └── tcga_blood_example.png
```
**⚠️ DO NOT CHANGE THIS CONFIGURATION, AS WE WILL RERUN YOUR NOTEBOOK USING THIS EXACT STRUCTURE!**

By running the following cells you will display a HE histological sample of a colorectal cancer case. It is composed of 3 main entities:
* **Mucin**: Grayish mucus that is secreted by the body. The presence of a tumor tends to increase its presence. 
* **Tumor**: Dark purple aggregates (hematoxylin). Mainly composed of cancerous cells.
* **Other**: Mixture of other cells such as stromal or lymphocytes. It appears mainly pink (eosin) but is dotted with nuclei (purple).

In [ ]:
img_he = show_introduction_figure()


# Part 1 - Segmentation [16 pts]

## Part 1.1 - RGB (3 pts)

**Q1 (1 pts)**: In this section, you will have to complete the function `extract_rgb_channels`. The function should extract and return red, blue, and green channels from the input image `img`. Your function will be fed to `plot_colors_histo` to plot the distribution of the colors in the image. 

In [ ]:
def extract_rgb_channels(img):
    """
    Extract RGB channels from the input image.

    Args
    ----
    img: np.ndarray (M, N, C)
        Input image of shape MxN and C channels.
    
    Return
    ------
    data_red: np.ndarray (M, N)
        Red channel of input image
    data_green: np.ndarray (M, N)
        Green channel of input image
    data_blue: np.ndarray (M, N)
        Blue channel of input image
    """

    # Get the shape of the input image
    M, N, _ = np.shape(img)

    # Define default values for RGB channels
    data_red = np.zeros((M, N))
    data_green = np.zeros((M, N))
    data_blue = np.zeros((M, N))

    data_red = img[:, :, 0]
    data_green = img[:, :, 1]
    data_blue = img[:, :, 2] 

    return data_red, data_green, data_blue

In [ ]:
################################################################
############################ TEST ##############################
################################################################

plot_colors_histo(
    img = img_he,
    func = extract_rgb_channels,
    labels = ["Red", "Green", "Blue"],
)


plt.figure(figsize=(12, 7))
plt.hist(rgb2gray(img_he).ravel(), bins=256)
plt.title('grayscale histogram')
plt.show()

R,G,B = extract_rgb_channels(img_he)

plt.figure(figsize=(12, 7))
plt.hist(R.ravel(), bins=256)
plt.title('red value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(G.ravel(), bins=256)
plt.title('green value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(B.ravel(), bins=256)
plt.title('blue histogram')
plt.show()

Based on the result of the plot above

* **Q2 (1 pts)**: Do you think you can find a simple manual thresholding approach to isolate the tumor components? (Justify)
    * **Answer**:The grayscale values obtained are not very discriminative. However, it is clearly observed that light purple pixels and dark purple pixels accumulate at grayscale values of 0.78 and 0.40, respectively. One can try thresholding to segment the image. Therefore, manual thresholding is applicable but not very effective. Also, note that the simple thresholding method masks the background from objects. Since some dark purple pixels appear as noise in other cells, thresholding alone cannot effectively mask those pixels.
* **Q3 (1 pts)**: Implement your own manual thresholding in `apply_rgb_threshold` to estimate the tumor location. Use the function `plot_thresholded_image` below to display your estimation results. Do you think your estimation is reliable? (justify)
    * **Answer**: Mucin in the image appears to be masked since its grayscale value is closer to "1" (assuming grayscale values are normalized). After applying the thresholding operation, those areas are shown as black because their grayscale value is higher than "0.41". In addition, the light purple pixels (other cells) are also shown as black. The darker pixels remain visible, and tumor cells become somewhat detectable but not very clear. Also, note that since there are dark purple pixels present in other cells, they remain visible, resulting in an image with white noise. The resulting estimation will not lead to false negatives, making it not entirely unreliable, but will introduce many false positives, making it impractical.

In [ ]:
def apply_rgb_threshold(img):
    """
    Apply threshold to input image. (R, G, B)

    Args
    ----
    img: np.ndarray (M, N, C)
        Input image of shape MxN and C channels.    
    Return
    ------
    img_th: np.ndarray (M, N)
        Thresholded image.
    """
    
    
    # r_low 
    # r_high
    # g_low
    # g_high
    # b_low
    # b_high

    thres_low = 0.41
    thresh_high = 1

    # Define the default value for the input image
    M, N, C = np.shape(img)
    img_th = np.zeros((M, N))
    img_th = rgb2gray(img)

    # Use the previous function to extract RGB channels
    data_red, data_green, data_blue = extract_rgb_channels(img=img)
    

    # r_mask =  data_red >= r_low & data_red <= r_high
    # g_mask =  data_green >= g_low & data_green <= g_high
    # b_mask =  data_blue >= b_low & data_blue <= b_high
    
    # mask = r_mask & g_mask & b_mask
    mask = (img_th >= thres_low) & (img_th <= thresh_high)
    img_th[ mask ] = 0 
    return  img_th

In [ ]:
################################################################
############################ TEST ##############################
################################################################

# Plot best RGB thresholding
plot_thresholded_image(img=img_he, func=apply_rgb_threshold, title="Best RGB Threshold")

## Part 1.2 - Other colorspaces (3 pts)

So far we used the standard RGB colorspace to apply our thresholding. In this section, you will convert the image to a different color space. 

* **Q1 (1 pts)**: Use the function `rgb2hsv` from the skimage package ([see doc](https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_rgb_to_hsv.html)) to convert the input image from RGB to HSV in function `extract_hsv_channels`.
* **Q2 (1 pts)**: Can you see any difference between the use of the RGB or HSV space? (justify)
    * **Answer**: The correlation between the pixel values is altered. In RGB, pixels are highly correlated, as observed in scatter plots where color intensity values show a strong linear relationship. In contrast, in the HSV space, the correlation structure changes significantly—hue, saturation, and value describe colors in a way that is closer to human perception. Notably, hue is less linearly correlated with the other channels, as it represents color independent of intensity. By using the HSV space, we should obtain better image segmentation.

In [ ]:
def extract_hsv_channels(img):
    """
    Extract HSV channels from the input image.

    Args
    ----
    img: np.ndarray (M, N, C)
        Input image of shape MxN and C channels.
    
    Return
    ------
    data_h: np.ndarray (M, N)
        Hue channel of input image
    data_s: np.ndarray (M, N)
        Saturation channel of input image
    data_v: np.ndarray (M, N)
        Value channel of input image
    """

    # Get the shape of the input image
    M, N, C = np.shape(img)

    img = rgb2hsv(img)

    # Define default values for HSV channels
    data_h = np.zeros((M, N))
    data_s = np.zeros((M, N))
    data_v = np.zeros((M, N))

    data_h = img[:, :, 0]
    data_s = img[:, :, 1]
    data_v = img[:, :, 2]

    return data_h, data_s, data_v

In [ ]:
################################################################
############################ TEST ##############################
################################################################

# Call plotting function with your implemented function
plot_colors_histo(
    img = img_he,
    func = extract_hsv_channels,
    labels = ["Hue", "Saturation", "Value"],
)

H,S,V = extract_hsv_channels(img_he)

plt.figure(figsize=(12, 7))
plt.hist(H.ravel(), bins=256)
plt.title('hue value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(S.ravel(), bins=256)
plt.title('saturation value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(V.ravel(), bins=256)
plt.title('value histogram')
plt.show()

* **Q3 (1 pts)**: Based on your results, try again to find the best manual threshold in function `apply_hsv_threshold`. Do you think your estimation is reliable? (justify)
    * **Answer**: As mentioned, HSV is a better representation for image segmentation. However, since running the expectation algorithm to fit a Gaussian mixture model with two Gaussians is not available, detecting a perfect threshold is not possible. Furthermore, the results are similar to those obtained with segmentation in the RGB space.

In [ ]:
def apply_hsv_threshold(img):
    """
    Apply threshold to the input image in hsv colorspace.

    Args
    ----
    img: np.ndarray (M, N, C)
        Input image of shape MxN and C channels.
    
    Return
    ------
    img_th: np.ndarray (M, N)
        Thresholded image.
    """

    #Fill in HSV threshold 0-1
    th_low = 270/360
    th_high = 310/360

    ts_low = 0.34
    ts_high = 0.47

    tv_low = 0.40
    tv_high = 0.50


    # Define the default value for the input image
    M, N, C = np.shape(img)
    #img_th = np.zeros((M, N))
    img_th = img.copy()

    # Use the previous function to extract HSV channels
    data_h, data_s, data_v = extract_hsv_channels(img=img)

    h_mask =  (data_h >= th_low) & (data_h <= th_high)
    s_mask =  (data_s >= ts_low) & (data_s <= ts_high)
    v_mask =  (data_v >= tv_low) & (data_v <= tv_high)


    #mask = h_mask & s_mask & v_mask
    mask = h_mask &  s_mask | v_mask
    
    img_th[ np.logical_not(mask) ] = 0


    return  rgb2gray(img_th)

In [ ]:
################################################################
############################ TEST ##############################
################################################################

# Find threshold(s) in the hsv channels
img_th=apply_hsv_threshold(img_he)

plot_thresholded_image(img=img_he, func=apply_hsv_threshold, title="Threshold in HSV color space")

## Part 1.3 - Morphology (5 pts)

To proceed, we will use your results from the previous thresholding (namely `apply_hsv_threshold`) as the starting point. In this exercise, we will try to clean the masked images using morphology to get a better estimation of the tumor area.

* **Q1 (1 pts)**: Implement the functions `apply_closing` and `apply_opening` with operations `closing` ([see doc](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.closing)), `opening` ([see doc](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.opening)) using the disk sizes arguments ([see doc](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.disk)).
* **Q2 (1 pts)**: We test disk sizes: $1, 2, 5, 10$. Comment on the quality of the results.
    * **Answer**: In our case, closing produces better results than opening because, after thresholding, the image contains small holes and gaps within the contours, which should not be present. Closing (dilation followed by erosion) effectively fills these unwanted holes, whereas opening worsens them. Beyond size 2, opening completely removes almost all objects. 
    Disk size 1 is too small to have a noticeable effect. Small holes and noise remain. Disk size 2 is optimal, as small holes are removed while preserving the tumor's shape. Larger sizes merge objects that should remain separate.

In [ ]:
def apply_closing(img_th, disk_size):
    """
    Apply closing to input mask image using disk shape.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    disk_size: int
        Size of the disk to use for opening

    Return
    ------
    img_closing: np.ndarray (M, N)
        Image after closing operation
    """

    # Define default value for output image
    img_closing = np.zeros_like(img_th)

    img_closing = closing(img_th, disk(disk_size))

    return img_closing


def apply_opening(img_th, disk_size):
    """
    Apply opening to input mask image using disk shape.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    disk_size: int
        Size of the disk to use for opening

    Return
    ------
    img_opening: np.ndarray (M, N)
        Image after opening operation
    """

    # Define default value for output image
    img_opening = np.zeros_like(img_th)

    img_opening = opening(img_th, disk(disk_size))

    return img_opening


In [ ]:
################################################################
############################ TEST ##############################
################################################################

def plot_close_open(img_th, apply_closing, apply_opening):
    disk_size = [1, 2, 5, 10]
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    for i, size in enumerate(disk_size):
        img_closing = apply_closing(img_th, size)
        img_opening = apply_opening(img_th, size)
        axs[0, i].imshow(img_closing, cmap="gray")
        axs[0, i].set_title(f"Closing with disk size {size}")
        axs[1, i].imshow(img_opening, cmap="gray")
        axs[1, i].set_title(f"Opening with disk size {size}")
    plt.show()

plot_close_open(img_th, apply_closing, apply_opening)


* **Q3 (1 pts)** Implement the functions `remove_holes` and `remove_objects` using operations [remove_small_holes](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.remove_small_holes), [remove_small_objects](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.remove_small_objects) from skimage package, respectively.
* **Q4 (1 pts)** We test the functions using area sizes: $10, 50, 100$, and $500$. Comment on the quality of the results.
    * **Answer**: Removing small holes smoothed the tumor's interior by filling unwanted gaps, with an area size of 100 providing the best balance. Smaller sizes (10, 50) left some holes, while 500 was too aggressive, over-smoothing holes that should remain . Similarly, removing small objects effectively eliminated unwanted healthy cell structures that remained after thresholding, with 100 being the most effective size. While it left some unwanted objects, 500 was too aggressive, removing parts of the tumor tissue.

In [ ]:
def remove_holes(img_th, size):
    """
    Remove holes from input image that are smaller than size argument.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    size: int
        Minimal size of holes

    Return
    ------
    img_holes: np.ndarray (M, N)
        Image after remove holes operation
    """

    # Define default value for input image
    img_th = img_th > 0

    img_holes = remove_small_holes(img_th, size)

    return img_holes


def remove_objects(img_th, size):
    """
    Remove objects from input image that are smaller than size argument.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    size: int
        Minimal size of objects

    Return
    ------
    img_obj: np.ndarray (M, N)
        Image after remove small objects operation
    """

    # Define default value for input image
    img_th = img_th > 0

    img_obj = remove_small_objects(img_th, size)

    return img_obj


In [ ]:
################################################################
############################ TEST ##############################
################################################################

def plot_remove_holes_objects(img, remove_holes, remove_objects):
    area_sizes = [10, 50, 100, 500]
    fig, axs = plt.subplots(2, len(area_sizes), figsize=(20, 10))
    for i, size in enumerate(area_sizes):
        img_holes = remove_holes(img, size)
        img_obj = remove_objects(img, size)
        axs[0, i].imshow(img_holes, cmap="gray")
        axs[0, i].set_title(f"Remove holes with size {size}")
        axs[1, i].imshow(img_obj, cmap="gray")
        axs[1, i].set_title(f"Remove objects with size {size}")
    plt.show()

#plot_remove_holes_objects(close_img, remove_holes, remove_objects)
plot_remove_holes_objects(img_th, remove_holes, remove_objects)


* **Q5 (1 pts)** Based on your previous results, implement the function `apply_morphology` that combines morphology functions to improve your tumor detection results from HSV thresholding. In this exercise, we prioritize having a higher number of false positives rather than false negatives, as mistakenly labeling non-tumor regions as tumors (over-segmentation) is preferable to missing actual tumor regions, which could lead to undetected cancerous areas and delayed treatment.

In [ ]:
def apply_morphology(img_th):
    """
    Apply morphology to thresholded image

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.

    Return
    ------
    img_morph: np.ndarray (M, N)
        Image after morphological operations
    """
    img_morph = np.zeros_like(img_th)
    img_th = apply_closing(img_th, 2)
    img_th[img_th > 0] = 1
    img_th = img_th.astype(bool)
    img_morph = remove_holes(img_th, 100)
    img_morph = remove_objects(img_morph, 500)
    img_morph = apply_opening(img_morph, 2)
    #img_morph = apply_closing(img_morph, 2)
    
    return img_morph

In [ ]:
################################################################
############################ TEST ##############################
################################################################

img_best_morpho = apply_morphology(apply_hsv_threshold(img_he))
plot_morphology_best(
    img_source=img_he,
    img_best=img_best_morpho,
)

## Part 1.4 - Region Growing (4 pts)

* **Q1 (3 pts)** In this section we ask you to implement from scratch your own `region_growing` function to detect the tumor area. The function should take as input an RGB image, seeds that are manual initialization points in the tumor area, and the maximum number of iterations to perform. Note that the function also includes the kwargs argument. You can use it to add additional parameters to your function if needed (see [doc kwargs](https://book.pythontips.com/en/latest/args_and_kwargs.html)). Importantly, for timing reasons, a single explicit for loop on iterations is expected. An explicit for loop on pixels or neighbours will be accepted but penalised by 1 point. Use numpy tools to succeed.
* **Q2 (1 pts)** Your function is then used to perform region growing using multiple iterations. Comment on the quality of the results based on the number of iterations performed as well as the running time.
    * **Answer**: As we increase the number of iterations, the region identified as the tumor gets bigger. The basic algorithm starts from seed points and checks neighboring pixels to see if their values vary within a given threshold. If they do, those pixels become the next active points (the same procedure applied to the initial points is then applied to these active points), while the ones used in the previous iteration are no longer considered. As the maximum number of iterations increases, more of the region gets covered. Thanks to this method elimination of white noise is possible, and only connected objects are indicated at the end of the algorithm.


In [ ]:
def region_growing(
    seeds: list[tuple],
    img: np.ndarray,
    n_max: int = 10,
    **kwargs
):
    """
    Run region growing on input image using seed points.

    Args
    ----
    seeds: list of tuple
        List of seed points
    img: np.ndarray (M, N, C)
        RGB image of size M, N, C
    n_max: int
        Number maximum of iterations before stopping algorithm

    Return
    ------
    rg: np.ndarray (M, N)
        Image after region growing has been performed
    """

    
    threshold = kwargs['threshold']

    if(not kwargs['isGrayScale']):
        gray_img = rgb2gray(img)
    else:
        gray_img = img
    
    M,N = gray_img.shape
    rg = np.zeros((M, N)).astype(bool)
    seeds_numpy = np.array(seeds)
    seed_intensity = gray_img[seeds_numpy[:,0],seeds_numpy[:,1]]
    queue = np.zeros((M, N), dtype=bool)
    queue[seeds_numpy[:, 0], seeds_numpy[:, 1]] = True
    neighbor_offsets = np.array([
        [-1, 0], [1, 0], [0, -1], [0, 1]  # Up, Down, Left, Right
    ])

    for i in range(n_max):
        active_pixels = np.argwhere(queue)

        if active_pixels.size == 0:
            break  # Stop if no more pixels to process

        # Reset queue
        queue.fill(False)

        # Get neighboring positions
        neighbors = (active_pixels[:, None, :] + neighbor_offsets).reshape(-1, 2)

        # Keep only valid positions
        valid_mask = (
            (neighbors[:, 0] >= 0) & (neighbors[:, 0] < M) &
            (neighbors[:, 1] >= 0) & (neighbors[:, 1] < N)
        )
        neighbors = neighbors[valid_mask]

        # Compute intensity difference
        neighbor_intensity = gray_img[neighbors[:, 0], neighbors[:, 1]]
        diff = np.abs(neighbor_intensity - seed_intensity.mean())

        # Select pixels within the threshold and not already part of the region
        valid_neighbors = neighbors[(diff <= threshold) & (~rg[neighbors[:, 0], neighbors[:, 1]])]


        # Update region and queue
        rg[valid_neighbors[:, 0], valid_neighbors[:, 1]] = True
        queue[valid_neighbors[:, 0], valid_neighbors[:, 1]] = True

    
    # ------------------
    # Your code here ... 
    # ------------------
                    
    return rg

In [ ]:
# Add you additional arguments in the dictionary below
kwargs = {'threshold':0.1,'isGrayScale':False}
# region_growing([], np.ones((5,5,3)), 10, **kwargs)

In [ ]:
################################################################
############################ TEST ##############################
################################################################

img_grow = plot_tumor_region_growing(img_he, region_growing, **kwargs)

## Part 1.5 - Final Comparison (1 pts)

* **Q1 (1 pts)** Run the cell below. Based on your observation, which approach do you think gives the best estimation of the tumor area? (justify)
    * **Answer**: Simple thresholding is not capable of filtering high frequency noise. One can use morphology after thresholding image in HSV space and region growing algorithms to encapsulate the targeted region more precisely.

In [ ]:
################################################################
############################ TEST ##############################
################################################################

plot_final_comparison(img_he, img_th, img_best_morpho, img_grow)

---
# Part 2 - Sandbox [8 pts]

For the second part, you will work on a new HE case without help. This time we ask you to
* **Q1 (4 pts)**: detect and compute the area of the blood cells,
* **Q2 (4 pts)**: detect and compute the area of the mucin.

Provide your results to the function `plot_results`. See the example below. Be careful, the completely white area is not mucin. It is the slide background and should be discarded. Moreover, you can focus on the large blood cell aggregates (large red areas)

In [ ]:
img_he2 = show_exo2_figure()

In [ ]:
M, N, C = np.shape(img_he2)
mask_blood = np.zeros((M, N))
mask_mucin = np.zeros((M, N))

H,S,V = extract_hsv_channels(img_he2)

plt.figure(figsize=(12, 7))
plt.hist(H.ravel(), bins=256)
plt.title('hue value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(S.ravel(), bins=256)
plt.title('saturation value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(V.ravel(), bins=256)
plt.title('value histogram')
plt.show()

R,G,B = extract_rgb_channels(img_he2)

plt.figure(figsize=(12, 7))
plt.hist(R.ravel(), bins=256)
plt.title('red value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(G.ravel(), bins=256)
plt.title('green value histogram')
plt.show()

plt.figure(figsize=(12, 7))
plt.hist(B.ravel(), bins=256)
plt.title('blue histogram')
plt.show()


In [ ]:
# simple thresholding in RGB space
mask_blood = (img_he2[:,:,0]>248) & (img_he2[:,:,1]<190) & (img_he2[:,:,2]<190)

# closing and removing small objects, since we focus on the large blood cell aggregates
mask_blood = apply_closing(mask_blood, 3)
mask_blood = remove_objects(mask_blood, 90)

# making the blood pixels white
img_he2_copy = img_he2.copy()
img_he2_copy[mask_blood] = 255

gray_img = rgb2gray(img_he2_copy)

plt.imshow(gray_img)
plt.title('Gray scale of the image after masking blood pixels as white pixels')

plt.figure(figsize=(12, 7))
plt.hist(gray_img.ravel(), bins=256)
plt.title('gray_image histogram after masking blood pixels as white pixels')
plt.show()


# mask white pixels to black
mask_white = gray_img[:,:]>0.9
gray_img[mask_white] = 0

plt.imshow(gray_img)
plt.title('Gray scale of the image after masking white pixels as black pixels')


# Find locations and corresponding grayscale values
mask_almost_mucin = gray_img[:,:]>0.899
some_mucin_pixel_locations = np.argwhere(mask_almost_mucin)
grayscale_values = gray_img[mask_almost_mucin]  # Extract pixel values

# Sort by grayscale intensity in descending order
sorted_indices = np.argsort(grayscale_values)[::-1]  # Sort and reverse order

# Select the top 4 brightest pixels
num_pixels = len(sorted_indices)
top_4_indices = sorted_indices[:min(4, num_pixels)]  # Take top 4
selected_locations_2D = some_mucin_pixel_locations[top_4_indices]

# Plot the grayscale image
plt.figure(figsize=(10, 10))
plt.imshow(gray_img, cmap="gray")
plt.scatter(selected_locations_2D[:, 1], selected_locations_2D[:, 0], 
            color='red', marker='o', s=100, label="Top 4 Brightest Pixels")
plt.legend()
plt.title("Top 4 Brightest Pixels in Mucin Region")
plt.show()

# making the location tuple list to put into function region_growing
list_tuple_locations = []
for i in range(len(selected_locations_2D)):
    list_tuple_locations.append(tuple(selected_locations_2D[i]))



In [ ]:
################################################################
############################ TEST ##############################
################################################################

# To fine tune region growing easily function result and hyperparameters are obtained and selected respectively here
kwargs = {'threshold':0.15,'isGrayScale':True}
mask_mucin = region_growing(list_tuple_locations,gray_img,2000,**kwargs)

plot_results(img=img_he2, mask_blood=mask_blood, mask_mucin=mask_mucin)